# Book Recommender for Quiz System

This notebook is demonstrate the book recommender system that used in Quiz platform. The recommendation is based on how much a student is performing in particular subject. For example if student is prforming bad on biology, then the recommender system is returning lists of biology book. For now, its just recommending book, and can be expanded to recommending other learning material such as journal or blog.

Author : Muhammad Asaduddin

# 1. Load Data and Library

## Load Libraries
----
We use pandas library to processing the data easily, especially for csv files. we use Sklearn to turn string categories into number categories, and numpy to create numpy array

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

## Load Data

### Download Data
We use Book data from Indonesian's kemendikbudristek (Ministry of Education, Culture, Research and Technology), for both textbooks, and non-textbooks. We also need to download or define the lookup table for Id

In [2]:
!curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/text_books.csv
!curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/non_text_books.csv
# !curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/publishers.csv
# !curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/text_book_school_levels.csv
# !curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/text_book_school_classes.csv
# !curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/text_book_subjects.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2026k  100 2026k    0     0  1607k      0  0:00:01  0:00:01 --:--:-- 1608k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5276k  100 5276k    0     0  4456k      0  0:00:01  0:00:01 --:--:-- 4459k


In [3]:
book_df = pd.read_csv('./text_books.csv')
# subjects = pd.read_csv('./text_book_subjects.csv')
# classes = pd.read_csv('./text_book_school_classes.csv')
# publishers = pd.read_csv('./publishers.csv')

In [4]:
book_df

,id,isbn,nuib,title,author,edition,publication_year,description,subject_id,classification_id,class_id,school_level_id,publisher_id,prices,physical_description,synopsis,category_id,category_path,category_is_leaf
0,004acdf2-2e34-4d78-a132-33c5bda6561c,NaN,BTPL00001-000220-4126,Selalu Berhemat Energi 4B Teks Tematik Terpadu...,Dhiah Saptorini; Agus Wahyudi,NaN,NaN,NaN,935a14ff-1e68-4419-99b4-bdad47b2cb13,c8c14d0e-7bca-4c50-95c3-e6cc68c710f1,d109dcf2-60a6-42c9-9450-0a77baa0d79f,41a81c2a-166f-400d-b9a1-f6cf0c973e1f,054db149-8fce-40f9-b92e-a4810fa757b8,"[{""id"":""22631ca0-b021-4d5c-8d99-6a38bf5913a8"",...","{""size"":""A4"",""width"":21,""weight"":"""",""page_cont...",NaN,1618,"""Barang"",""Buku Teks"",""Buku Siswa"",""Jenjang SD/...",True
1,009074b7-a413-4183-8965-7ff242d018a3,NaN,BTPL00002-014020-0185,Buku Guru Tema 4 Keluargaku 1 Untuk SD/MI Kela...,Annisa Nur Rohmah; Istiana Nen Arienti; Slamet...,NaN,NaN,NaN,7784f0b2-2422-47be-aeef-76904e8e9631,c617a25f-f013-423d-a5f6-16be3a8d89d3,91d98fdf-e385-4102-8938-64af720718c9,41a81c2a-166f-400d-b9a1-f6cf0c973e1f,9157eb7e-9c1a-4b9d-aeed-727fd98fc9af,"[{""id"":""cc6bafb0-86f9-4dd3-b5dd-b30c94ee931d"",...","{""size"":""B5"",""width"":17.5,""weight"":"""",""page_co...",NaN,1760,"""Barang"",""Buku Teks"",""Buku Guru"",""Jenjang SD/M...",True
2,011f0232-f1ad-4aff-80b1-da70b2321825,NaN,BTPL00003-005120-8549,"Buku Siswa Pendidikan Jasmani, Olahraga, dan K...",Sardianto,NaN,NaN,NaN,7c35f50b-4f39-473b-9096-5f7e3474e3ff,c8c14d0e-7bca-4c50-95c3-e6cc68c710f1,e84c73cd-57c5-4442-b856-c7fdba0752df,6bdd711e-9a16-4449-bacf-a08f1879b36b,3c493ae1-f62e-41f4-9479-28b035715a10,"[{""id"":""415a467b-4b6e-465d-8edd-e80a78ad44a7"",...","{""size"":""B5"",""width"":17.5,""weight"":"""",""page_co...",NaN,1667,"""Barang"",""Buku Teks"",""Buku Siswa"",""Jenjang SMP...",True
3,013ba3b3-d7c6-4ea7-a9aa-f98778ef3bf6,978-602-244-696-5,BTPL00854-045823-3968,Buku Kurikulum Merdeka_Buku Panduan Guru Pendi...,"Susi Bonardy, Yenny Suria",1.0,2021.0,NaN,62233bc3-50e9-40b0-8221-d379e91c388d,c617a25f-f013-423d-a5f6-16be3a8d89d3,e4b14f52-715b-44d2-a82f-17e09dabef4f,41a81c2a-166f-400d-b9a1-f6cf0c973e1f,c1a471cf-53d9-483d-b958-d200724ef17e,"[{""id"":""98850c06-bf9e-4f0b-ab7f-c10ff97f1bdf"",...","{""size"":""17,6 x 25"",""width"":17,""weight"":"""",""pa...",NaN,1768,"""Barang"",""Buku Teks"",""Buku Guru"",""Jenjang SD/M...",True
4,01554a5e-9f62-435d-9a93-d8e099d11783,NaN,BTPL00005-045820-5298,Tema 7: Perkembangan Teknologi,NaN,NaN,2017.0,Untuk SD/MI Kelas 3,6145713e-b201-4cdb-b784-a7764abcad52,c8c14d0e-7bca-4c50-95c3-e6cc68c710f1,cc4b9872-a57f-4ca0-9945-70d794196fbc,41a81c2a-166f-400d-b9a1-f6cf0c973e1f,c1a471cf-53d9-483d-b958-d200724ef17e,"[{""id"":""a0518541-8c70-428e-ace4-f0074d262b73"",...","{""size"":""210 x 297"",""width"":"""",""weight"":"""",""pa...",NaN,1609,"""Barang"",""Buku Teks"",""Buku Siswa"",""Jenjang SD/...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,fee113d6-ee36-4d7d-b054-3a979c3cffe5,978-602-244-856-3,BTPL00831-045823-4293,Buku Kurikulum Merdeka_Buku Panduan Guru Sejar...,"Martina Safitry, Indah Wahyu Puji Utami, Zein ...",1.0,2021.0,NaN,7a60ed58-60f5-4bf0-a724-140f194e0493,c617a25f-f013-423d-a5f6-16be3a8d89d3,4807005f-e2cc-4ec1-8b27-4fbe09d72690,b5092d4e-c5a5-4f3d-89ac-70661366eac7,c1a471cf-53d9-483d-b958-d200724ef17e,"[{""id"":""98850c03-21df-46a8-84d7-b5c815b587fc"",...","{""size"":""17,6 x 25"",""width"":17,""weight"":"""",""pa...",NaN,1911,"""Barang"",""Buku Teks"",""Buku Guru"",""Jenjang SMA/...",True
1490,ff04fc84-db33-4ee0-ad1a-b06476dc6ccf,978-602-244-448-0,BTPL00770-045823-7906,Buku Kurikulum Merdeka_Pendidikan Pancasila da...,"Muhammad Sapei, Trezadigjaya, Prayogo",1.0,2021.0,NaN,9a653172-ed0d-4c03-99b5-2d88d4cadcaa,c8c14d0e-7bca-4c50-95c3-e6cc68c710f1,86ea3e63-6928-41cb-a75a-40852e2cca96,6bdd711e-9a16-4449-bacf-a08f1879b36b,c1a471cf-53d9-483d-b958-d200724ef17e,"[{""id"":""98850bf7-1e65-4b59-a941-3d84b8306762"",...","{""size"":""17,6 x 25"",""width"":17,""we

# 2. Cleaning The Data

## Create new Dataframe to just include important metadata
---- 
The Previous DataFrame contains all information about the book. It contains datas like author, publisher, year, price, and other. Because we recommend book based on quizes result, we don't need any metadata beside school level, class level, and subject. To make efficient data frame, we make another dataframe to just contain important data that we need for recommender. The reason we made a new data frame is that to make sure if we mess up the data frame, we can just go back without re-load the entire data from csv.


While investigate the data, we notice there is a column named `category_path` which contain all important data like subject, school level, and class level. We decide to use that column and turn that column into different columns


The steps to make new data frame to recommendation are:
1. Make helper function to split data from `catgory_path` column and a function to remove class level in subject
2. Include only the important columns from previous data frame
3. Apply helper function to clean data
4. Remove the books that for teacher and PAUD level
5. Standarize the similar subject name
6. Turn school_level, class_level, and subject to categorical

## Helper Function and Variables
Helper Function to accelerate preprocessing data, and dictionary to turn string category into numbers

In [5]:
# turn category_path values into different columns 
def preprocess_value(x: str) -> list:
    #repalce double ticks, 'Jenjang', and 'Kelas' into empty string
    x = x.replace('"', '')
    x = x.replace('Jenjang ', '')
    x = x.replace('Kelas ', '')
    #split string in comma sign
    x_list = x.split(',')
    return x_list

# remove specific string in the list. This used to get rid of class level on category column
def remove_string_in_list(x: str, remove_list: list[str]) -> str:
    # create list contain number from 00 to 10
    num_list = [ f'0{i}' if i <= 9 else f'{i}' for i in range(13)]
    # copy the list of string that we tryng to remove
    stopword = remove_list.copy()
    # extend the remove list to include list that contain numbers
    stopword.extend(num_list)
    # get rid of dash symbol, and pre and post spaces
    x = x.replace('-', '')
    x = x.strip()
    # split string into list, and begin to remove the string from the list
    x_list = x.split(' ')
    x_list = [word for word in x_list if word not in stopword]
    x_list = [subject_similar[word] if word in subject_similar.keys() else word for word in x_list]
    # join the list to build the entire subject
    x = ' '.join(x_list)
    x = x.strip()
    return x

In [6]:
class_map = {
    'PAUD' : 0.0,
    '1' : 1.0,
    '2' : 2.0,
    '3' : 3.0,
    '4' : 4.0,
    '5' : 5.0,
    '6' : 6.0,
    '7' : 7.0,
    '8' : 8.0,
    '9' : 9.0,
    '10': 10.0,
    '11': 11.0,
    '12': 12.0,
    'Semua Kelas' : 13.0
}

school_list = {
    'PAUD' : 0.0,
    'SD/MI' : 1.0,
    'SMP/MTs' : 2.0,
    'SMA/MA' : 3.0,
    'SMK/MAK' : 4.0,
    'SMA/SMK' : 5.0,
    'SMA/MA - SMK/MAK' : 6.0,
    'SDLB - SMPLB - SMA/KLB' : 7.0 
}

subject_similar = {
    'Pendidikan Agama Budha': 'Pendidikan Agama Buddha',
    'PJOK' : 'Pendidikan Jasmani Olahraga dan Kesehatan',
}

In [7]:
train_df = book_df.loc[:, ['id', 'title']]
columns = ['usage', 'school_level', 'class_level', 'subject']

train_df[columns] = book_df['category_path'].apply(
    lambda x: pd.Series(preprocess_value(x)[2:])
)
school_level = list(school_list.keys())
train_df['subject'] = train_df.subject.map(lambda x: remove_string_in_list(x, school_level))
train_df = train_df[train_df['usage'] != 'Buku Guru']
train_df = train_df[train_df['class_level'] != 'PAUD'] 
train_df.reset_index(drop=True, inplace=True)
train_df.subject = train_df.subject.apply(lambda x: subject_similar[x] if x in subject_similar.keys() else x)
# book_df_new = book_df_new.dropna()
# book_df_new = book_df_new.reset_index().loc[:,'id':'subject']

In [8]:
train_df

,id,title,usage,school_level,class_level,subject
0,004acdf2-2e34-4d78-a132-33c5bda6561c,Selalu Berhemat Energi 4B Teks Tematik Terpadu...,Buku Siswa,SD/MI,4,Tema 2
1,011f0232-f1ad-4aff-80b1-da70b2321825,"Buku Siswa Pendidikan Jasmani, Olahraga, dan K...",Buku Siswa,SMP/MTs,7,Pendidikan Jasmani Olahraga dan Kesehatan
2,01554a5e-9f62-435d-9a93-d8e099d11783,Tema 7: Perkembangan Teknologi,Buku Siswa,SD/MI,3,Tema 7
3,01c7db56-2e52-4a30-8749-282a94e2b7ff,"Tema 7 Benda, Hewan, dan Tanaman di Sekitarku ...",Buku Siswa,SD/MI,1,Tema 7
4,01e4effa-ec64-49f1-b4de-939ae8754d80,Ilmu Pengetahuan Alam,Buku Siswa,SMP/MTs,7,IPA
...,...,...,...,...,...,...
770,fe4634a3-31d8-421e-b609-c261839ef705,"Tema 6 Lingkungan Bersih, Sehat, dan Asri Buku...",Buku Siswa,SD/MI,1,Tema 6
771,ff04fc84-db33-4ee0-ad1a-b06476dc6ccf,Buku Kurikulum Merdeka_Pendidikan Pancasila da...,Buku Siswa,SMP/MTs,8,Pendidikan Pancasila
772,ff1db59b-6036-475a-920a-c7cc95289e00,Bahasa Indonesia,Buku Siswa,SMA/MA,12,Bahasa Indonesia
773,ff43461c-3ddc-4931-9760-ed5b3be4c3c5,Pendidikan Agama Islam dan Budi Pekerti,Buku Siswa,SMP/MTs,7,Pendidikan Agama Islam


## Turn String category to number category

In [9]:
enc = OneHotEncoder(sparse=False)
enc.fit(train_df[['subject']])
columns = [j for items in enc.categories_ for j in items]
train_df[columns] = enc.transform(train_df[['subject']])

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
#train_df.drop(['school_level','class_level','subject'], axis=1, inplace=True)
# enc.inverse_transform([[1, 45]])
train_df.school_level = train_df.school_level.map(lambda x: school_list[x])
train_df.class_level = train_df.class_level.map(lambda x: class_map[x])
train_df.drop(['subject'], axis=1, inplace=True)

In [11]:
train_df

,id,title,usage,school_level,class_level,Agribisnis Perikanan,Agribisnis Tanaman,Agribisnis Ternak,Agriteknologi Pengolahan Hasil Pertanian,Akuntansi dan Keuangan Lembaga,...,Tema 2,Tema 3,Tema 4,Tema 5,Tema 6,Tema 7,Tema 8,Tema 9,Usaha Layanan Pariwisata,Usaha Pertanian Terpadu
0,004acdf2-2e34-4d78-a132-33c5bda6561c,Selalu Berhemat Energi 4B Teks Tematik Terpadu...,Buku Siswa,1.0,4.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,011f0232-f1ad-4aff-80b1-da70b2321825,"Buku Siswa Pendidikan Jasmani, Olahraga, dan K...",Buku Siswa,2.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01554a5e-9f62-435d-9a93-d8e099d11783,Tema 7: Perkembangan Teknologi,Buku Siswa,1.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,01c7db56-2e52-4a30-8749-282a94e2b7ff,"Tema 7 Benda, Hewan, dan Tanaman di Sekitarku ...",Buku Siswa,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,01e4effa-ec64-49f1-b4de-939ae8754d80,Ilmu Pengetahuan Alam,Buku Siswa,2.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,fe4634a3-31d8-421e-b609-c261839ef705,"Tema 6 Lingkungan Bersih, Sehat, dan Asri Buku...",Buku Siswa,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
771,ff04fc84-db33-4ee0-ad1a-b06476dc6ccf,Buku Kurikulum Merdeka_Pendidikan Pancasila da...,Buku Siswa,2.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
772,ff1db59b-6036-475a-920a-c7cc95289e00,Bahasa Indonesia,Buku Siswa,3.0,12.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
773,ff43461c-3ddc-4931-9760-ed5b3be4c3c5,Pendidikan Agama Islam dan Budi Pekerti,Buku Siswa,2.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 3. Create User Data

## Create user data via pandas data frame

In this section we are going to create user data to build our recommender system. For example only, we create user data using pandas dataframe. We define user id, quiz's school level, quiz's class level, quiz's suject, and score.

*note: On real world application, or in deployment, we use user's history of quizes that stored in database. In database, we create result table wich contain userId, quizId, and result. While the quiz's school level, quiz class level, and quiz subject is in different table called quizes that store entire quiz metadata*

In [12]:
user_data = pd.DataFrame([
    ['1234', 3, 11, 31, 90],
    ['3211', 1, 4, 31, 80],
    ['1234', 3, 11, 13, 60],
    ['1234', 3, 12, 6, 60],
    ['3211', 1, 7, 31, 20],
    ['1234', 5, 12, 31, 70],
    ['3211', 1, 5, 6, 60],
    ['1234', 6, 10, 13, 80],
    ['3211', 1, 4, 6, 80],
], columns=['user_id', 'school_level', 'class_level', 'subject', 'score'])

user_data

,user_id,school_level,class_level,subject,score
0,1234,3,11,31,90
1,3211,1,4,31,80
2,1234,3,11,13,60
3,1234,3,12,6,60
4,3211,1,7,31,20
5,1234,5,12,31,70
6,3211,1,5,6,60
7,1234,6,10,13,80
8,3211,1,4,6,80


In [13]:
user_matrix = pd.DataFrame(np.zeros((1,79)))
user_matrix[[0,1]] = user_data.loc[user_data['user_id'] == '1234', ['school_level','class_level']].mean()
# user_matrix[1] = user_data.loc[user_data['user_id'] == '1234', 'class_level'].mean()
for i in range(77):
    user_matrix[i+2] = 50 / user_data.loc[(user_data['user_id'] == '1234') & (user_data['subject'] == i), 'score'].mean()
user_matrix.fillna(0, inplace = True)
# user_matrix

/tmp/ipykernel_19/887852007.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  user_matrix[[0,1]] = user_data.loc[user_data['user_id'] == '1234', ['school_level','class_level']].mean()


In [14]:
user_matrix.loc[:,2:].transpose()

,0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
...,...
74,0.0
75,0.0
76,0.0
77,0.0


In [15]:
distances = train_df.iloc[:,5:].dot(user_matrix.loc[:,2:].transpose().to_numpy())
distances.sort_values(0, ascending=False)[:20]
# np.argmax(distances)

,0
644,0.833333
587,0.833333
753,0.833333
676,0.833333
556,0.833333
60,0.833333
218,0.833333
106,0.833333
34,0.833333
743,0.833333


In [16]:
train_df.loc[644]

id                                       cd7a7f17-ee5f-6ddc-7247-ee275603d02b
title                       Buku Kurikulum Merdeka_Bahasa Indonesia: Kawan...
usage                                                              Buku Siswa
school_level                                                              1.0
class_level                                                               3.0
                                                  ...                        
Tema 7                                                                    0.0
Tema 8                                                                    0.0
Tema 9                                                                    0.0
Usaha Layanan Pariwisata                                                  0.0
Usaha Pertanian Terpadu                                                   0.0
Name: 644, Length: 82, dtype: object